## Introduction

In this Notebook, we will code and backtest an RSI Trading Strategy. RSI is a Technical Indicator used to find oversold and overbought prices and therefore is popular for mean reversion strategies

## Strategy Description

We will trade on the EURUSD daily timeframe. When the RSI indicator drops below 30 (oversold), we will buy and when RSI exceeds above 70 (overbought), we will sell.

For Exit, we will use the ATR Indicator. ATR measures the average range for each period and is therefore a good volatility indicator. After entering a trade, we will set take profit and stop loss to 2 ATR each

---

In [13]:
# Libraries 
import MetaTrader5 as mt5
import pandas as pd
from IPython.display import FileLink, FileLinks
import plotly.express as px

In [14]:
# connect to MetaTrader5 as mt5
mt5.initialize()

True

In [15]:
# settings
symbol = 'EURUSD'
timeframe = mt5.TIMEFRAME_D1
start_pos = 0
num_bars = 1000

## Retrieving Historical Prices

In [16]:
# Requesting historical data
bars = mt5.copy_rates_from_pos(symbol, timeframe, start_pos, num_bars)
df = pd.DataFrame(bars)[['time', 'open', 'high', 'low', 'close']]
df['time'] = pd.to_datetime(df['time'], unit='s')

df.to_csv('outputs/data.csv', index=False)
df.to_excel('outputs/data.xlsx', index=False)

df

,time,open,high,low,close
0,2019-05-27,1.12094,1.12150,1.11865,1.11908
1,2019-05-28,1.11907,1.12009,1.11586,1.11591
2,2019-05-29,1.11594,1.11721,1.11240,1.11302
3,2019-05-30,1.11306,1.11430,1.11157,1.11283
4,2019-05-31,1.11287,1.11795,1.11248,1.11676
...,...,...,...,...,...
995,2023-03-24,1.08320,1.08384,1.07124,1.07601
996,2023-03-27,1.07706,1.08001,1.07451,1.07977
997,2023-03-28,1.07974,1.08486,1.07934,1.08444
998,2023-03-29,1.08454,1.08717,1.08178,1.08444


## Plotting EURUSD Close Prices

In [17]:
fig = px.line(df, x='time', y='close', title='EURUSD - Close Prices')  # creating a figure using px.line
display(fig)  # showing figure in output

## Calculating RSI Indicator

To find details how the indicator is calculated, watch this video: https://youtu.be/vT0-eLOw5Uk

In [18]:
# setting the RSI Period
rsi_period = 14

# to calculate RSI, we first need to calculate the exponential weighted aveage gain and loss during the period
df['gain'] = (df['close'] - df['open']).apply(lambda x: x if x > 0 else 0)
df['loss'] = (df['close'] - df['open']).apply(lambda x: -x if x < 0 else 0)

# here we use the same formula to calculate Exponential Moving Average
df['ema_gain'] = df['gain'].ewm(span=rsi_period, min_periods=rsi_period).mean()
df['ema_loss'] = df['loss'].ewm(span=rsi_period, min_periods=rsi_period).mean()

# the Relative Strength is the ratio between the exponential avg gain divided by the exponential avg loss
df['rs'] = df['ema_gain'] / df['ema_loss']

# the RSI is calculated based on the Relative Strength using the following formula
df['rsi_14'] = 100 - (100 / (df['rs'] + 1))

# displaying the results
display(df[['time', 'rsi_14', 'rs', 'ema_gain', 'ema_loss']])

# plotting the RSI
fig_rsi = px.line(df, x='time', y='rsi_14', title='RSI Indicator')

# RSI commonly uses oversold and overbought levels, usually at 70 and 30
overbought_level = 70
orversold_level = 30

# adding oversold and overbought levels to the plot
fig_rsi.add_hline(y=overbought_level, opacity=0.5)
fig_rsi.add_hline(y=orversold_level, opacity=0.5)

# showing the RSI Figure
display(fig_rsi)

,time,rsi_14,rs,ema_gain,ema_loss
0,2019-05-27,NaN,NaN,NaN,NaN
1,2019-05-28,NaN,NaN,NaN,NaN
2,2019-05-29,NaN,NaN,NaN,NaN
3,2019-05-30,NaN,NaN,NaN,NaN
4,2019-05-31,NaN,NaN,NaN,NaN
...,...,...,...,...,...
995,2023-03-24,53.510861,1.151040,0.002950,0.002563
996,2023-03-27,56.779418,1.313712,0.002918,0.002221
997,2023-03-28,62.110333,1.639242,0.003156,0.001925
998,2023-03-29,61.922828,1.626245,0.002735,0.001682


## Calculating ATR Indicator

In [19]:
atr_period = 14  # defining the atr period to 14

# calculating the range of each candle
df['range'] = df['high'] - df['low']

# calculating the average value of ranges
df['atr_14'] = df['range'].rolling(atr_period).mean()

display(df[['time', 'atr_14']])

# plotting the ATR Indicator
fig_atr = px.line(df, x='time', y='atr_14', title='ATR Indicator')
display(fig_atr)

,time,atr_14
0,2019-05-27,NaN
1,2019-05-28,NaN
2,2019-05-29,NaN
3,2019-05-30,NaN
4,2019-05-31,NaN
...,...,...
995,2023-03-24,0.010874
996,2023-03-27,0.010206
997,2023-03-28,0.010246
998,2023-03-29,0.010250


## Importing classes for Backtesting

In [20]:
# class Position contain data about trades opened/closed during the backtest 
class Position:
    def __init__(self, open_datetime, open_price, order_type, volume, sl, tp):
        self.open_datetime = open_datetime
        self.open_price = open_price
        self.order_type = order_type
        self.volume = volume
        self.sl = sl
        self.tp = tp
        self.close_datetime = None
        self.close_price = None
        self.profit = None
        self.status = 'open'
        
    def close_position(self, close_datetime, close_price):
        self.close_datetime = close_datetime
        self.close_price = close_price
        self.profit = (self.close_price - self.open_price) * self.volume if self.order_type == 'buy' \
                                                                        else (self.open_price - self.close_price) * self.volume
        self.status = 'closed'
        
    def _asdict(self):
        return {
            'open_datetime': self.open_datetime,
            'open_price': self.open_price,
            'order_type': self.order_type,
            'volume': self.volume,
            'sl': self.sl,
            'tp': self.tp,
            'close_datetime': self.close_datetime,
            'close_price': self.close_price,
            'profit': self.profit,
            'status': self.status,
        }
        

# class Strategy defines trading logic and evaluates the backtest based on opened/closed positions
class Strategy:
    def __init__(self, df, starting_balance):
        self.starting_balance = starting_balance
        self.positions = []
        self.data = df
    
    # return backtest result
    def get_positions_df(self):
        df = pd.DataFrame([position._asdict() for position in self.positions])
        df['pnl'] = df['profit'].cumsum() + self.starting_balance
        return df
    
    # add Position class to list
    def add_position(self, position):
        self.positions.append(position)
        return True
    
    # close positions when stop loss or take profit is reached
    def close_tp_sl(self, data):
        for pos in self.positions:
                    if pos.status == 'open':
                        if (pos.sl >= data.close and pos.order_type == 'buy'):
                            pos.close_position(data.time, pos.sl)
                        elif (pos.sl <= data.close and pos.order_type == 'sell'):
                            pos.close_position(data.time, pos.sl)
                        elif (pos.tp <= data.close and pos.order_type == 'buy'):
                            pos.close_position(data.time, pos.tp)
                        elif (pos.tp >= data.close and pos.order_type == 'sell'):
                            pos.close_position(data.time, pos.tp)
                            
    # check for open positions
    def has_open_positions(self):
        for pos in self.positions:
            if pos.status == 'open':
                return True
        return False
    
    # strategy logic how positions should be opened/closed
    def logic(self, data):
        
        # if no position is open
        if not self.has_open_positions():
            
            # if RSI less then 30 -> BUY
            if data['rsi_14'] < 30:
                
                # Position variables
                open_datetime = data['time']
                open_price = data['close']
                order_type = 'buy'
                volume = 10000
                sl = open_price - 2 * data['atr_14']
                tp = open_price + 2 * data['atr_14']
                
                self.add_position(Position(open_datetime, open_price, order_type, volume, sl, tp))
            
            # if RSI greater than 70 -> SELL
            elif data['rsi_14'] > 70:
                
                # Position variables
                open_datetime = data['time']
                open_price = data['close']
                order_type = 'sell'
                volume = 10000
                sl = open_price + 2 * data['atr_14']
                tp = open_price - 2 * data['atr_14']
                
                self.add_position(Position(open_datetime, open_price, order_type, volume, sl, tp))
        
        
# logic
    def run(self):
        # data represents a moment in time while iterating through the backtest
        for i, data in self.data.iterrows():
            # close positions when stop loss or take profit is reached
            self.close_tp_sl(data)
            
            # strategy logic
            self.logic(data)
        
        return self.get_positions_df()

In [21]:
# preparing data for backtest
backtest_df = df[14:]  # removing NaN values
backtest_df

,time,open,high,low,close,gain,loss,ema_gain,ema_loss,rs,rsi_14,range,atr_14
14,2019-06-14,1.12761,1.12888,1.12020,1.12103,0.00000,0.00658,0.001421,0.002038,0.697044,41.074020,0.00868,0.006259
15,2019-06-17,1.12088,1.12467,1.12033,1.12178,0.00090,0.00000,0.001343,0.001736,0.773973,43.629344,0.00434,0.006267
16,2019-06-18,1.12171,1.12425,1.11809,1.11936,0.00000,0.00235,0.001147,0.001826,0.628341,38.587811,0.00616,0.006364
17,2019-06-19,1.11926,1.12538,1.11868,1.12259,0.00333,0.00000,0.001462,0.001562,0.935989,48.346812,0.00670,0.006647
18,2019-06-20,1.12255,1.13173,1.12252,1.12916,0.00661,0.00000,0.002197,0.001339,1.640615,62.130035,0.00921,0.006914
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-03-24,1.08320,1.08384,1.07124,1.07601,0.00000,0.00719,0.002950,0.002563,1.151040,53.510861,0.01260,0.010874
996,2023-03-27,1.07706,1.08001,1.07451,1.07977,0.00271,0.00000,0.002918,0.002221,1.313712,56.779418,0.00550,0.010206
997,2023-03-28,1.07974,1.08486,1.07934,1.08444,0.00470,0.00000,0.003156,0.001925,1.639242,62.110333,0.00552,0.010246
998,2023-03-29,1.08454,1.08717,1.08178,1.08444,0.00000,0.00010,0.002735,0.001682,1.626245,61.922828,0.00539,0.010250


## Running the backtest

In [22]:
# creating an instance of Strategy class
rsi_strategy = Strategy(backtest_df, 10000)

# running the backtest
backtest_result = rsi_strategy.run()

backtest_result

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2019-06-21,1.13699,sell,10000,1.150713,1.123267,2019-07-05,1.123267,137.228571,closed,10137.228571
1,2019-07-09,1.12078,buy,10000,1.110093,1.131467,2019-07-31,1.110093,-106.871429,closed,10030.357143
2,2019-07-31,1.10749,buy,10000,1.097169,1.117811,2019-08-05,1.117811,103.214286,closed,10133.571429
3,2019-08-19,1.10778,buy,10000,1.094779,1.120781,2019-09-25,1.094779,-130.014286,closed,10003.557143
4,2019-10-16,1.10710,sell,10000,1.117766,1.096434,2019-12-30,1.117766,-106.657143,closed,9896.900000
5,2019-12-30,1.11985,sell,10000,1.129821,1.109879,2020-01-17,1.109879,99.714286,closed,9996.614286
6,2020-01-24,1.10255,buy,10000,1.093571,1.111529,2020-02-10,1.093571,-89.785714,closed,9906.828571
7,2020-02-10,1.09105,buy,10000,1.082046,1.100054,2020-02-18,1.082046,-90.042857,closed,9816.785714
8,2020-02-18,1.07907,buy,10000,1.069737,1.088403,2020-02-27,1.088403,93.328571,closed,9910.114286
9,2020-02-27,1.09998,sell,10000,1.110979,1.088981,2020-03-02,1.110979,-109.985714,closed,9800.128571


## Visualizing the Backtest

In [23]:
# analysing closed positions only
backtest_result = backtest_result[backtest_result['status'] == 'closed']

# visualizing trades
fig_backtest = px.line(df, x='time', y=['close'], title='RSI Strategy - Trades')

# adding trades to plots
for i, position in backtest_result.iterrows():
    if position.status == 'closed':
        fig_backtest.add_shape(type="line",
            x0=position.open_datetime, y0=position.open_price, x1=position.close_datetime, y1=position.close_price,
            line=dict(
                color="green" if position.profit >= 0 else "red",
                width=3)
            )

fig_backtest

## Plotting PnL

In [24]:
fig_pnl = px.line(backtest_result, x='close_datetime', y='pnl')
fig_pnl